In [2]:
import time
import requests
from hashlib import sha1
import random
import string
import json

headers = {'Accept': 'application/vnd.booli-v2+json'}
callerId = "nordstromcarolin.mau"
limit = 100                    #Denna ska vara 500 sedan när vi kör på riktigt.
offset = 0                    #denna måste vi uppdatera med 500 för varje varv.
#bbox = "55.571166,12.908381,55.593296,12.985119"
#bboxbell = "55.600793,12.939270,55.585710,12.952099"
bbox_ny = "55.58132,12.91899,55.59570,12.97305"

sold_object_list = []

for page_num in range(28):
    timestamp = str(int(time.time()))
    unique = ''.join(random.choice(string.ascii_uppercase + string.digits) for x in range(16))
    hashstr = sha1((callerId + timestamp + "uwu3Q63nDqI9D0k8MCaNE0QrRcZbibIB8Dkn0iD0" + unique).encode('utf-8')).hexdigest()
    offset = page_num * limit
    print('limit is', limit)
    print('offset is', offset)

    url = f"http://api.booli.se/sold?q=malmö&bbox={bbox_ny}&callerId={callerId}&time={timestamp}&unique={unique}&hash={hashstr}&limit={limit}&offset={offset}"
    print(url)
    response = requests.get(url, headers=headers)

    if response.status_code == 429:
        print('Rate limited! Please wait.')
        time.sleep(int(response.headers['retry-after']))
        continue
    if(response.status_code != 200):
        print("fail")

    result = json.loads(response.text)
    print(result['totalCount'])
    sold_object_list.extend(result['sold'])
    print("------")
    wait = random.randint(500,3000)/1000
    print("wait ", wait)
    time.sleep(wait)


limit is 100
offset is 0
http://api.booli.se/sold?q=malmö&bbox=55.58132,12.91899,55.59570,12.97305&callerId=nordstromcarolin.mau&time=1654590012&unique=KCO1E1JMSGJTF1QL&hash=5b5850351ada2081554cea5e935dc2ab11fe1327&limit=100&offset=0
2898
------
wait  1.145
limit is 100
offset is 100
http://api.booli.se/sold?q=malmö&bbox=55.58132,12.91899,55.59570,12.97305&callerId=nordstromcarolin.mau&time=1654590013&unique=B6I68YW06CM2Q4AC&hash=d0f337bb1bd98db577a307603dbbc11f294dcbdc&limit=100&offset=100
2898
------
wait  2.651
limit is 100
offset is 200
http://api.booli.se/sold?q=malmö&bbox=55.58132,12.91899,55.59570,12.97305&callerId=nordstromcarolin.mau&time=1654590016&unique=4U50X7TZKD9ZPN0E&hash=58970a4c161566e88d6485d2dfedf5917f97cb3f&limit=100&offset=200
2898
------
wait  1.664
limit is 100
offset is 300
http://api.booli.se/sold?q=malmö&bbox=55.58132,12.91899,55.59570,12.97305&callerId=nordstromcarolin.mau&time=1654590018&unique=1QI2C7UP1WP1G2UC&hash=7d8d6edd30d2c14423e10c1b76442bda844534bb&l

In [3]:
len(sold_object_list)

2800

In [4]:
#prints the data in json. Fixes the å,ä,ö

print(json.dumps(sold_object_list[1], indent=2, ensure_ascii=False)) 


{
  "source": {
    "id": 12,
    "url": "http://www.vaningen.se/",
    "type": "Broker",
    "name": "Våningen & Villan"
  },
  "rooms": 3,
  "constructionYear": 1948,
  "soldDate": "2022-06-06",
  "location": {
    "address": {
      "city": "Malmö",
      "streetAddress": "Norra Klockspelsvägen 5C"
    },
    "position": {
      "latitude": 55.595387,
      "longitude": 12.965079
    },
    "region": {
      "municipalityName": "Malmö",
      "countyName": "Skåne län"
    },
    "distance": {
      "ocean": 772
    },
    "namedAreas": [
      "Fridhem"
    ]
  },
  "additionalArea": 0,
  "rent": 5146,
  "livingArea": 81,
  "soldPriceSource": "maklare",
  "listPrice": 5395000,
  "soldPrice": 5395000,
  "booliId": 4657513,
  "objectType": "Radhus",
  "published": "2022-05-26 16:25:49",
  "url": "https://www.booli.se/annons/4657513"
}


In [8]:
#Here the attributes that we want to use is taken out.
#Also the specific adress "streetAddress" and "namedArea"


specific_attributes = []
constYear = []
for att in sold_object_list:
    try:
        street = att['location']['address']['streetAddress']
        area = att['location']['namedAreas'][0]
        constYear = att['constructionYear'] 
        objectType = att['objectType']
        specific_attributes.append([street, area, constYear, objectType])
    except KeyError:
        pass
    
print(json.dumps(specific_attributes, indent=4, ensure_ascii=False))
print(len(specific_attributes))

[
    [
        "Fernissagatan 8",
        "Limhamns Sjöstad",
        2019,
        "Lägenhet"
    ],
    [
        "Norra Klockspelsvägen 5C",
        "Fridhem",
        1948,
        "Radhus"
    ],
    [
        "Köpenhamnsvägen 46B",
        "Slottsstaden",
        1954,
        "Lägenhet"
    ],
    [
        "Caritasgatan 16",
        "Bellevue Sjösida",
        1926,
        "Villa"
    ],
    [
        "Bagaregatan 3",
        "Limhamn",
        1910,
        "Parhus"
    ],
    [
        "Skarhultsgatan 8",
        "Rosenvång",
        1936,
        "Villa"
    ],
    [
        "Lokstallsgatan 31",
        "Limhamns Sjöstad",
        2015,
        "Radhus"
    ],
    [
        "Valborgsgatan 44",
        "Limhamn",
        2006,
        "Radhus"
    ],
    [
        "Ynglingagatan 30B",
        "Bellevue Sjösida",
        1967,
        "Villa"
    ],
    [
        "Geijersgatan 64",
        "Nya Bellevue",
        1962,
        "Villa"
    ],
    [
        "Västanväg 59C",
  

In [14]:
len(specific_attributes)

2487

In [13]:
#Saves the list to a csv-file

import pandas as pd

df = pd.DataFrame(specific_attributes)
df.columns = ['Adress', 'Område', 'Byggår', 'Typ av byggnad'] 
df.to_csv('booli_2.csv', index=False, encoding='utf-8-sig')
